In [1]:
tstart_utc="2023-05-21T05:30:43.000000"
duration_days=10.01
target_loc="healpix"
target_ra=83.01
target_dec=22.02
target_radius=1.
target_healpix_url="https://gracedb.ligo.org/api/superevents/S230521k/files/bayestar.multiorder.fits,1"
enable_esac=1

In [2]:
from astropy.table import QTable
from astropy import units as u
import astropy_healpix as ah
import numpy as np
import integralvisibility
from matplotlib import pylab
from astropy.coordinates import SkyCoord


<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


failed to get AUTH with env due to: 'K8S_SECRET_INTEGRAL_CLIENT_SECRET'
got AUTH with homefile for integral-limited


In [3]:
skymap = QTable.read(target_healpix_url)
skymap

import ligo.skymap.moc
import healpy

d = ligo.skymap.moc.rasterize(skymap, order=8)

healpy.write_map('skymap.fits', d['PROBDENSITY'], nest=True, overwrite=True)

healpy.mollview(d['PROBDENSITY'], nest=True)


2023-05-22 08:46:25 savchenk-NG-VN7-793G-79EG healpy[255854] WARNING setting the output map dtype to [dtype('float64')]


In [4]:
target_mp = healpy.read_map('skymap.fits')
healpy.mollview(target_mp)

2023-05-22 08:46:26 savchenk-NG-VN7-793G-79EG healpy[255854] INFO NSIDE = 256
2023-05-22 08:46:26 savchenk-NG-VN7-793G-79EG healpy[255854] INFO ORDERING = NESTED in fits file
2023-05-22 08:46:26 savchenk-NG-VN7-793G-79EG healpy[255854] INFO INDXSCHM = IMPLICIT
2023-05-22 08:46:26 savchenk-NG-VN7-793G-79EG healpy[255854] INFO Ordering converted to RING


In [5]:
import integralclient as ic
import datetime

if tstart_utc == "unknown":
    tstart_utc = datetime.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S")

tstart_ijd = float(ic.converttime("UTC",tstart_utc,"IJD"))

tstop_ijd = tstart_ijd+duration_days

2023-05-22 08:46:26 savchenk-NG-VN7-793G-79EG root[255854] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/UTC/2023-05-21T05:30:43.000000/IJD


In [6]:
# %matplotlib notebook

In [7]:
vis = integralvisibility.Visibility(minsolarangle=40)
vis.minsolarangle=40

2023-05-22 08:46:27 savchenk-NG-VN7-793G-79EG root[255854] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/UTC/2002-01-01T00:00:00/IJD
2023-05-22 08:46:28 savchenk-NG-VN7-793G-79EG root[255854] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/UTC/2012-01-01T00:00:00/IJD


In [8]:
mp=vis.for_time(tstart_ijd,nsides=128)



2023-05-22 08:46:29 savchenk-NG-VN7-793G-79EG root[255854] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/ANY/8541.2304650925925671/IJD


get grid...
sun coord <SkyCoord (ICRS): (ra, dec) in deg
    (56.07413579, 19.80857065)>
will compute map.. (196608,)


In [9]:
sc = ic.get_sc(tstart_utc)
sc

{'barytime': -0.13589770447981409,
 'bodies': {'earth': {'body_dec': -72.895,
   'body_distance': 0,
   'body_in_sc': [11.196984595473598, 20.45467056593822],
   'body_ra': 18.39300000000003,
   'body_size': 2.4989297373600805,
   'integral_parallax': 57.29577951308232,
   'separation': 146075.1},
  'moon': {'body_dec': -72.895,
   'body_distance': 393026.586400955,
   'body_in_sc': [89.14433913949102, 1.5873702799135723],
   'body_ra': 18.39300000000003,
   'body_size': 0.2638112104851575,
   'integral_parallax': 22.1855204103284,
   'separation': 377249.96155848855},
  'sun': {'body_dec': -72.895,
   'body_distance': 151392379.32343245,
   'body_in_sc': [70.31969893085207, -0.5990881464681543],
   'body_ra': 18.39300000000003,
   'body_size': 0.26332301764346894,
   'integral_parallax': 0.05528954453725959,
   'separation': 151375577.28136575}},
 'd_scx': 11.1969845954736,
 'd_scy': 86.10890065768447,
 'd_scz': 79.51727123608072,
 'dyz_scy': 69.54532943406177,
 'dyz_scz': 20.45467056

In [10]:
sc['bodies']['earth']['body_ra']

18.39300000000003

In [11]:
now_point = SkyCoord(sc['scx']['ra'], sc['scx']['dec'], unit='deg')
now_point

<SkyCoord (ICRS): (ra, dec) in deg
    (352.233, -8.09)>

In [12]:
import requests

#http://integral.esac.esa.int/isocweb/tvp.html?revolution=2050&action=perRevolution
r=requests.get("http://integral.esac.esa.int/isocweb/tvp.html",
            params=dict(
                    startDate=tstart_utc[8:10]+"-"+tstart_utc[5:7]+"-"+tstart_utc[:4],                    
                    duration="12.600",
                    dither="R",
                    action="predict",
                    endDate="31-12-2019",
                    coordinates="equatorial",
                    ra=target_ra,
                    dec=target_dec,
                    format="json"))

try:
    if not enable_esac: raise RuntimeError
    
    visibility = r.json()
    
    #visibility
    pylab.figure()

    for interval in visibility['INTERVALS'][:10]:
        print(interval)
        i1=float(ic.converttime("UTC",interval['start'][:-4].replace(" ","T"),"IJD"))
        i2=float(ic.converttime("UTC",interval['end'][:-4].replace(" ","T"),"IJD"))

        print(i1,i2)

        pylab.axvspan(i1,i2,alpha=0.5)

    pylab.axvspan(tstart_ijd,tstop_ijd,alpha=0.5,color='g')
    pylab.xlabel("IJD")

    pylab.savefig('visibility_intervals.png')

except:
    print(r.text)









<html>
<head>
    <title>Target Visibility Predictor</title>
    <link type="text/css" rel="stylesheet" href="javascript/js9support.css">
    <link type="text/css" rel="stylesheet" href="javascript/js9.css">

    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
    <script type="text/javascript" src="javascript/js9prefs.js"></script>
    <script type="text/javascript" src="javascript/js9support.min.js"></script>
    <script type="text/javascript" src="javascript/js9.min.js"></script>
    <script type="text/javascript" src="javascript/js9plugins.js"></script>
    <script type="text/javascript">
        function show() {

        }
    </script>
    <style type="text/css">
        #centerdiv {
            width: 100%;
        / / margin-left: auto;
        / / margin-right: auto;
        }

        #JS9ToolbarToolbar {
            width: 516px;
            height: 144px;
            min-height: 30px;
        }

        .JS9Toolba

In [13]:
import requests

esac_visibility_rev = ic.converttime('UTC',tstart_utc,'REVNUM')

print("will fetch ESOC visibility map for rev",esac_visibility_rev)

try:
    r=requests.get("http://integral.esac.esa.int/isocweb/tvp.html",
            params=dict(
                    revolution=esac_visibility_rev,
                    action='perRevolution',
                    ))

    esac_visibility_map = r.json()
    print("ESAC VM success")
except:
    esac_visibility_map = None
    print("ESAC visibility failed:", r.text)

#visibility




2023-05-22 08:46:33 savchenk-NG-VN7-793G-79EG root[255854] INFO url https://www.astro.unige.ch/cdci/astrooda/dispatch-data/gw//timesystem/api/v1.0/converttime/UTC/2023-05-21T05:30:43.000000/REVNUM


will fetch ESOC visibility map for rev 2642
ESAC VM success


In [14]:
import healpy

import numpy as np

mp_ra, mp_dec = integralvisibility.healpy.pix2ang(128,range(integralvisibility.healpy.nside2npix(128)),lonlat=True)

esac_mp = np.zeros_like(mp_ra)

try:
    if not enable_esac: raise RuntimeError
    for sky_bin in esac_visibility_map['sky_bins'][:10000]:
        #print(sky_bin)

        m = (mp_ra > float(sky_bin['top_left_corner'][0])) & (mp_ra < float(sky_bin['top_right_corner'][0]))
        m &= (mp_dec < float(sky_bin['top_left_corner'][1])) & (mp_dec > float(sky_bin['bottom_left_corner'][1]))

        esac_mp[m]=float(sky_bin['duration'])
except Exception as e:
    print("ESAC visibility failed:", e)
    esac_mp = mp



In [18]:
import gzip

if target_loc == "healpix":
    from io import BytesIO
    from astropy.io import fits
   
    f = fits.open(target_healpix_url)
    
    healpy.mollview(target_mp)
    
    target_mp = healpy.get_interp_val(target_mp, mp_ra, mp_dec, lonlat=True)
    target_mp/=target_mp.sum()

    healpy.mollview(target_mp)
    
    indices = np.argsort(-target_mp)
    target_cum_map = np.empty(target_mp.shape)
    target_cum_map[indices] = 100 * np.cumsum(target_mp[indices])

    healpy.mollview(target_cum_map)

    extract={}
    
    for k,v in dict(f[1].header).items():    
        if k.startswith('LOG') or \
           k.startswith('DIST') or \
           k.startswith('INSTRUME'):
            extract[k] = v
            print(k,v)
else:
    extract={}
    
extract

INSTRUME H1,L1
DISTMEAN 453.9716641904466
DISTSTD 162.7030083768224
LOGBCI 1.700376464510647
LOGBSN 6.043954181283272


{'INSTRUME': 'H1,L1',
 'DISTMEAN': 453.9716641904466,
 'DISTSTD': 162.7030083768224,
 'LOGBCI': 1.700376464510647,
 'LOGBSN': 6.043954181283272}

In [19]:
def total_slew_time_s(distance_deg):
    total_slew_options = []
    
    slew_duration_s = distance_deg*60*60/185.
    
    total_slew_options.append(slew_duration_s)
    
    if distance_deg>45:
        rwb_duration_s = 22*60. 
        total_slew_options.append(slew_duration_s + rwb_duration_s)
    
    return total_slew_options

def total_reaction_time_s(distance_deg):
    return [ (x + 3600*2) for x in total_slew_time_s(distance_deg)]
    
total_reaction_time_s(10),total_reaction_time_s(50)

([7394.594594594595], [8172.972972972973, 9492.972972972973])

In [20]:
visibility_summary = {}

for kind, vismp in [("std", mp), ("esac", esac_mp)]:
    visibility_summary[kind]={}
    vis = visibility_summary[kind]
    
    nsides = healpy.npix2nside(len(vismp))

  #  integralvisibility.healpy.mollview(vismp,title="INTEGRAL visibility at "+tstart_utc[:19],cbar=False)
  #  integralvisibility.healpy.graticule()
  #  pylab.savefig("visibility_"+kind+".png")

    radec=integralvisibility.healpy.pix2ang(integralvisibility.healpy.npix2nside(len(vismp)), range(len(vismp)), lonlat=True)
    ra,dec=radec
    
    sep_from_now_deg = np.transpose(SkyCoord(*radec,unit='deg').separation(now_point).deg)
    
    dist_ra=(ra - target_ra)
    dist_ra[abs(dist_ra)>180]-=360

    if target_loc == "point":
        
            target_mp=np.exp(
                -0.5*(dist_ra/target_radius)**2  \
                -0.5*((dec - target_dec)/target_radius )**2
            )

            target_mp/=np.sum(target_mp)
        
      
        

    integralvisibility.healtics.plot_with_ticks((vismp/vismp.max())*100, 
                                                overplot=[[(target_mp,'r',target_mp.max()/10.)]],
                                                vmin=0,
                                                cmap="summer",
                                                unit="%",
                                                title="INTEGRAL visibility at "+tstart_utc[:19])

#     integralvisibility.healpy.mollview(vismp/vismp.max() + target_mp/target_mp.max(),title="INTEGRAL visibility at "+tstart_utc[:19],cbar=False)
#     integralvisibility.healpy.graticule()

    vis['on_peak'] = vismp[target_mp.argmax()]
    vis['on_peak_frac'] = vis['on_peak'] / vismp.max()

    vis['on_probability'] = target_mp[vismp > vismp.max()/2.].sum()

    #vis['visible_peak_ra'] = healpy.pix2ang(nsides, target_mp[vismp > vismp.max()/2.].argmax(),)
    vis['visible'] = vis['on_probability']
    
    def argmax_masked(x,y):
        mi = x[y].argmax()
        print("mi",mi)
        idx = np.array(range(len(x)))
        print("idx",idx)
        return idx[y][mi]

    vis['peak_of_target'] = list(np.transpose(radec)[target_mp.argmax()])
    vis['peak_of_visible'] = list(np.transpose(radec)[argmax_masked(target_mp, vismp > vismp.max()/2.)])


    vis['points'] = [
        dict(descr="total visble",prob=vis['on_probability']),
        dict(descr="peak",ra=vis['peak_of_target'][0],dec=vis['peak_of_target'][1]),
        dict(descr="visible peak",ra=vis['peak_of_visible'][0],dec=vis['peak_of_visible'][1]),
    ]
    
    
    colors=['r','m','k','b','g','c','y','w','grey']
    for limrange in None, 45, 90:
        
        limmod = "lim%.3lgdeg_"%limrange if limrange is not None else ""
        limdescr = " (%.3lg deg from now)"%limrange if limrange is not None else ""
        
        
        for scale, descr in [(5, "best for staring"), (15, "best for HEX"), (30, "best for 5x5")]:
            c=colors.pop()
            target_sm = healpy.smoothing(target_mp/target_mp.max(), sigma=scale/180.*np.pi,verbose=False)
            target_sm/=sum(target_sm)

            print(sum(target_sm),sum(target_mp))
                        
            mask_visible = (vismp > vismp.max()/2.)
            if limrange is not None:
                mask_visible &= (sep_from_now_deg < limrange)
                
            vis[limmod+'best_visible_%ideg'%scale]=list(np.transpose(radec)[argmax_masked(target_sm, mask_visible)])
            vis[limmod+'best_visible_prob_%ideg'%scale]=max(target_sm[mask_visible])*scale*scale*4*np.pi*2  

            true_peak = SkyCoord(vis['peak_of_target'][0],vis['peak_of_target'][1],unit="deg")

            vis['points'] += [dict(
                alt_descr="peak at %i deg scale"%scale+limdescr,            
                descr = descr+limdescr,            
                ra=vis[limmod+'best_visible_%ideg'%scale][0],
                dec=vis[limmod+'best_visible_%ideg'%scale][1],
                prob=vis[limmod+'best_visible_prob_%ideg'%scale],
                distance_to_true_peak_deg=SkyCoord(vis[limmod+'best_visible_%ideg'%scale][0], vis[limmod+'best_visible_%ideg'%scale][1],unit="deg").separation(true_peak).deg,
                distance_to_now=SkyCoord(vis[limmod+'best_visible_%ideg'%scale][0], vis[limmod+'best_visible_%ideg'%scale][1],unit="deg").separation(now_point).deg
            )]


            def c_ra(x):
                if x<180:
                    return -x
                return 360-x

            pylab.scatter(
                c_ra(vis[limmod+'best_visible_%ideg'%scale][0])/180.*np.pi,
                vis[limmod+'best_visible_%ideg'%scale][1]/180.*np.pi,
                s=100,c=c)

            pylab.text(
                c_ra((-5+vis[limmod+'best_visible_%ideg'%scale][0]))/180.*np.pi,
                (5+vis[limmod+'best_visible_%ideg'%scale][1])/180.*np.pi,
                "%i deg"%scale,
                size=10,color=c)
        
    
    pylab.scatter(
        c_ra(sc['scx']['ra'])/180.*np.pi,
        sc['scx']['dec']/180.*np.pi,
        marker='x',
        s=200,c='k',
        label='FoV')
    
    #pylab.legend()

    

        
    print(visibility_summary)


    pylab.savefig("skymap_visibility_"+kind+".png")

to overplot 1
to overplot [[(array([2.80725428e-05, 2.79947201e-05, 3.34719287e-05, ...,
       1.21846539e-08, 3.88842074e-08, 5.79106362e-08]), 'r', 1.4587878685421798e-05)]]
op set (array([2.80725428e-05, 2.79947201e-05, 3.34719287e-05, ...,
       1.21846539e-08, 3.88842074e-08, 5.79106362e-08]), 'r', 1.4587878685421798e-05)


2023-05-22 08:48:18 savchenk-NG-VN7-793G-79EG astropy[255854] WARNING AstropyDeprecationWarning: "verbose" was deprecated in version 1.15.0 and will be removed in a future version. 


overplotting [[3.12238785e-08 3.12238785e-08 3.12238785e-08 ... 3.12238785e-08
  3.12238785e-08 3.12238785e-08]
 [2.84190303e-08 2.84453687e-08 2.84717072e-08 ... 2.83663534e-08
  2.83926918e-08 2.84190303e-08]
 [2.56141821e-08 2.56668590e-08 2.57195359e-08 ... 2.55088282e-08
  2.55615051e-08 2.56141821e-08]
 ...
 [3.07330043e-05 3.07438105e-05 3.07546168e-05 ... 3.07113917e-05
  3.07221980e-05 3.07330043e-05]
 [3.07217448e-05 3.07271479e-05 3.07325510e-05 ... 3.07109385e-05
  3.07163416e-05 3.07217448e-05]
 [3.07104853e-05 3.07104853e-05 3.07104853e-05 ... 3.07104853e-05
  3.07104853e-05 3.07104853e-05]]
levels: [1.4587878685421798e-05]
mi 87410
idx [     0      1      2 ... 196605 196606 196607]


2023-05-22 08:48:19 savchenk-NG-VN7-793G-79EG healpy[255854] INFO Sigma is 300.000000 arcmin (0.087266 rad) 
2023-05-22 08:48:19 savchenk-NG-VN7-793G-79EG healpy[255854] INFO -> fwhm is 706.446014 arcmin
2023-05-22 08:48:19 savchenk-NG-VN7-793G-79EG healpy[255854] INFO Sigma is 0.000000 arcmin (0.000000 rad) 
2023-05-22 08:48:19 savchenk-NG-VN7-793G-79EG healpy[255854] INFO -> fwhm is 0.000000 arcmin


0.9999999999999831 1.0000000000000122
mi 87091
idx [     0      1      2 ... 196605 196606 196607]


...

0.9999999999999831 1.0000000000000122
mi 27737
idx [     0      1      2 ... 196605 196606 196607]


2023-05-22 08:48:29 savchenk-NG-VN7-793G-79EG healpy[255854] INFO Sigma is 900.000000 arcmin (0.261799 rad) 
2023-05-22 08:48:29 savchenk-NG-VN7-793G-79EG healpy[255854] INFO -> fwhm is 2119.338041 arcmin
2023-05-22 08:48:29 savchenk-NG-VN7-793G-79EG healpy[255854] INFO Sigma is 0.000000 arcmin (0.000000 rad) 
2023-05-22 08:48:29 savchenk-NG-VN7-793G-79EG healpy[255854] INFO -> fwhm is 0.000000 arcmin


1.0000000000000095 1.0000000000000122
mi 27175
idx [     0      1      2 ... 196605 196606 196607]


2023-05-22 08:48:30 savchenk-NG-VN7-793G-79EG healpy[255854] INFO Sigma is 1800.000000 arcmin (0.523599 rad) 
2023-05-22 08:48:30 savchenk-NG-VN7-793G-79EG healpy[255854] INFO -> fwhm is 4238.676081 arcmin
2023-05-22 08:48:30 savchenk-NG-VN7-793G-79EG healpy[255854] INFO Sigma is 0.000000 arcmin (0.000000 rad) 
2023-05-22 08:48:30 savchenk-NG-VN7-793G-79EG healpy[255854] INFO -> fwhm is 0.000000 arcmin


0.9999999999999938 1.0000000000000122
mi 25498
idx [     0      1      2 ... 196605 196606 196607]
{'std': {'on_peak': 0.0, 'on_peak_frac': 0.0, 'on_probability': 0.6441934683014302, 'visible': 0.6441934683014302, 'peak_of_target': [55.546875, -17.27000469497446], 'peak_of_visible': [69.9609375, -28.630989836924357], 'points': [{'descr': 'total visble', 'prob': 0.6441934683014302}, {'descr': 'peak', 'ra': 55.546875, 'dec': -17.27000469497446}, {'descr': 'visible peak', 'ra': 69.9609375, 'dec': -28.630989836924357}, {'alt_descr': 'peak at 5 deg scale', 'descr': 'best for staring', 'ra': 71.015625, 'dec': -28.291548862929403, 'prob': 0.047839745737926126, 'distance_to_true_peak_deg': 17.992583478125784, 'distance_to_now': 76.33237139814253}, {'alt_descr': 'peak at 15 deg scale', 'descr': 'best for HEX', 'ra': 71.3671875, 'dec': -27.95318688338115, 'prob': 0.17802038785437369, 'distance_to_true_peak_deg': 18.06295778774766, 'distance_to_now': 76.65431818771496}, {'alt_descr': 'peak at 30 

In [21]:
visibility_summary['esac']

{'on_peak': 0.0,
 'on_peak_frac': 0.0,
 'on_probability': 0.5539621752746701,
 'visible': 0.5539621752746701,
 'peak_of_target': [55.546875, -17.27000469497446],
 'peak_of_visible': [82.265625, -34.590574976176626],
 'points': [{'descr': 'total visble', 'prob': 0.5539621752746701},
  {'descr': 'peak', 'ra': 55.546875, 'dec': -17.27000469497446},
  {'descr': 'visible peak', 'ra': 82.265625, 'dec': -34.590574976176626},
  {'alt_descr': 'peak at 5 deg scale',
   'descr': 'best for staring',
   'ra': 81.56249999999999,
   'dec': -33.152887928619876,
   'prob': 0.04601703083562378,
   'distance_to_true_peak_deg': 28.25899419295428,
   'distance_to_now': 85.02847820671627},
  {'alt_descr': 'peak at 15 deg scale',
   'descr': 'best for HEX',
   'ra': 81.56249999999999,
   'dec': -31.738394244764436,
   'prob': 0.17143422459372273,
   'distance_to_true_peak_deg': 27.62548245832647,
   'distance_to_now': 85.18825050917143},
  {'alt_descr': 'peak at 30 deg scale',
   'descr': 'best for 5x5',
   

In [22]:
template="""
<html>
<head>
</head>
<body>

<table style="border-collapse: separate;border-spacing: 2px;">


<tr style="background-color:#ADD8E6">
<td align="center"></td>
<td align="center">RA</td>
<td align="center">Dec</td>
<td align="center">Probability</td>
<!--<td align="center">To Peak</td>-->
<td align="center">From current</td>
</tr>

{% for point in points %}

<tr>
<td style="text-align:center" >
<div data-toggle="{{ point.uid }}_loc_tooltip" title="{{ point.alt_descr }}" style="display:inline-block;margin-right:10px;">
{{ point.descr }}
</div>
</td>

{% if point.ra %}
<td style="text-align:center" >{{ point.ra | round(1) }}</td>
<td style="text-align:center" >{{ point.dec | round(1)}}</td>
{% else %}
<td style="text-align:center" ></td>
<td style="text-align:center" ></td>
{% endif %}

{% if point.prob %}
<td style="text-align:center" >{{ (100*point.prob) | round(1)}}%</td>
{% else %}
<td style="text-align:center" ></td>
{% endif %}

<!--
{% if point.distance_to_true_peak_deg %}
<td style="text-align:center" >{{ point.distance_to_true_peak_deg | round(1)}} deg</td>
{% else %}
<td style="text-align:center" ></td>
{% endif %}
-->


{% if point.distance_to_now %}
<td style="text-align:center" >{{ point.distance_to_now | round(1)}} deg</td>
{% else %}
<td style="text-align:center" ></td>
{% endif %}


</tr>

{% endfor %}

<tr style="background-color:#ADD8E6">
<td colspan=5 height="5px"></td>
</tr>


</table>

</body>
</html>
"""

from IPython.core.display import display, HTML
from jinja2 import Environment, BaseLoader

rtemplate = Environment(loader=BaseLoader).from_string(template)


data_html = rtemplate.render(points=visibility_summary['esac']['points'])

open("visibility_card.html","w").write(data_html)

display(HTML(data_html))

/tmp/ipykernel_255854/1599995600.py:73: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


,RA,Dec,Probability,To Peak,From current
total visble,,,55.4%,,
peak,55.5,-17.3,,,
visible peak,82.3,-34.6,,,
best for staring,81.6,-33.2,4.6%,28.3 deg,85.0 deg
best for HEX,81.6,-31.7,17.1%,27.6 deg,85.2 deg
best for 5x5,81.6,-27.6,34.6%,26.1 deg,85.7 deg
best for staring (45 deg from now),315.4,18.2,0.1%,104.7 deg,44.9 deg
best for HEX (45 deg from now),34.8,-25.3,6.8%,20.9 deg,44.0 deg
best for 5x5 (45 deg from now),34.8,-25.3,25.0%,20.9 deg,44.0 deg
best for staring (90 deg from now),81.6,-33.2,4.6%,28.3 deg,85.0 deg


In [23]:
visibility_fraction=visibility_summary['esac']['visible']

In [24]:
skymap_png="skymap_visibility_std.png"
skymap_esac_png="skymap_visibility_esac.png"
visibility_std="visibility_std.png"
visibility_esac="visibility_esac.png"
visibility_intervals_png='visibility_intervals.png'
visibility_summary=visibility_summary
visibility_fraction=visibility_fraction
visibility_card="visibility_card.html"
extract=extract